In [17]:
import ast
import yaml
from pprint import pprint
from collections import defaultdict

py2bs_type_dict = {
    'str': 'string',
    'int': 'integer',
    'float': 'real',
    'bool': 'boolean',
}
py2bs_type_dict = defaultdict(lambda: "NOTIMPLEMENTED", py2bs_type_dict)

name = 'train'
script_filename = f'brane_{name}.py'
dir = 'training'

with open(f"{dir}/{script_filename}", "r") as source:
    tree = ast.parse(source.read())

install_dependencies = ['numpy', 'pandas', 'matplotlib', 'scipy', 'scikit-learn', 'lightgbm']

functions = [element for element in tree.body if type(element) is ast.FunctionDef]
containeryml_dict = {
'name': dir,
'version': '1.0.0',
'kind': 'compute',
'dependencies': [
    'python3', 'python3-yaml', 'python3-pip'
],
'files': [script_filename],
'install': [f'pip3 install {x}' for x in install_dependencies],
'entrypoint': {
  'kind': 'task',
  'exec': script_filename
},
'actions': {
    function.name:{
      'command': {'args': [function.name]},
      'input': [{'type': py2bs_type_dict[arg.annotation.id], 'name': arg.arg} for arg in function.args.args],
      'output': [{'type': 'string', 'name': 'output'}],
    }
 for function in functions},
}

with open(f"{dir}/container.yaml", 'w') as file:
    yaml.dump(containeryml_dict, file)

In [75]:
# from as far as I get its kinda impossible to know what is and isn't part of Python's 'standard library' hence install_dependencies need to be manually specified

aliases = []
imports = [element for element in tree.body if type(element) is ast.Import]
import_froms = [element for element in tree.body if type(element) is ast.ImportFrom]
for foo in imports: 
    for bar in foo.names: aliases.append(bar.name)
for foo in import_froms: aliases.append(foo.module.split('.')[0])
aliases = list(set(aliases))
aliases